In [1]:
from huggingface_hub import login

# Paste your Hugging Face token here
login("your_token_here")

In [2]:
!pip install transformers accelerate bitsandbytes datasets peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("json", data_files="qa_dataset.json")

# Split into train and validation sets
dataset = dataset["train"].train_test_split(test_size=0.2)

train_data = dataset["train"]
val_data = dataset["test"]

print(f"Training samples: {len(train_data)}, Validation samples: {len(val_data)}")


Generating train split: 0 examples [00:00, ? examples/s]

Training samples: 518, Validation samples: 130


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load tokenizer
model_name = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model with 4-bit quantization
from transformers import BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

# Resize token embeddings for tokenizer
model.resize_token_embeddings(len(tokenizer))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Embedding(128256, 2048)

In [5]:
# Add pad_token if not already present
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})  # Add padding token

# Resize model embeddings to match tokenizer
model.resize_token_embeddings(len(tokenizer))


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(128257, 2048)

In [6]:
from peft import get_peft_model, LoraConfig, TaskType

# Configure LoRA for the model
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,  # Task type
    r=16,                            # LoRA rank
    lora_alpha=32,                   # Scaling factor
    target_modules=["q_proj", "v_proj"],  # Apply LoRA to specific modules
    lora_dropout=0.05,               # Dropout rate
    bias="none"
)

# Apply LoRA configuration to the model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # Check trainable parameters


trainable params: 1,703,936 || all params: 1,237,520,384 || trainable%: 0.1377


In [7]:
def preprocess_function(examples):
    # Combine instruction and input fields for the input sequence
    inputs = [
        instruction + " " + inp
        for instruction, inp in zip(examples["instruction"], examples["input"])
    ]

    # Tokenize inputs
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding="max_length"
    )

    # Tokenize outputs (targets)
    outputs = tokenizer(
        examples["output"],
        max_length=128,  # Shorter max length for targets
        truncation=True,
        padding="max_length"
    )

    # Add labels to model inputs, ensuring correct shape
    model_inputs["labels"] = outputs["input_ids"]

    # Ensure labels have the same length as input_ids by padding/truncating
    # This step ensures that the batch sizes match during loss calculation.
    pad_token_id = tokenizer.pad_token_id
    label_padding_length = len(model_inputs["input_ids"][0]) - len(model_inputs["labels"][0])

    if label_padding_length > 0:
        model_inputs["labels"] = [label + [pad_token_id] * label_padding_length for label in model_inputs["labels"]]
    elif label_padding_length < 0:
        model_inputs["labels"] = [label[:len(model_inputs["input_ids"][0])] for label in model_inputs["labels"]]

    return model_inputs

# Apply preprocessing to the datasets
tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_val = val_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/518 [00:00<?, ? examples/s]

Map:   0%|          | 0/130 [00:00<?, ? examples/s]

In [8]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [9]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./qlora_results",  # Output directory
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    learning_rate=2e-4,           # Learning rate
    per_device_train_batch_size=8,  # Batch size for training
    per_device_eval_batch_size=8,   # Batch size for evaluation
    num_train_epochs=3,           # Number of epochs
    save_total_limit=2,           # Limit total number of saved checkpoints
    predict_with_generate=True,   # Enable text generation during evaluation
    fp16=True,                    # Use mixed precision
    logging_dir="./qlora_logs",   # Directory for logs
    logging_steps=10,             # Log every 10 steps
    load_best_model_at_end=True,  # Load best model based on validation loss
    metric_for_best_model="eval_loss",
    save_strategy="epoch" # Change save_strategy to "epoch" to match evaluation_strategy
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

<ipython-input-10-171c91793687>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [11]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,11.267600,11.291985
2,11.171200,11.220528
3,11.179400,11.202380


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


TrainOutput(global_step=195, training_loss=11.429676114595853, metrics={'train_runtime': 722.0613, 'train_samples_per_second': 2.152, 'train_steps_per_second': 0.27, 'total_flos': 4653824945946624.0, 'train_loss': 11.429676114595853, 'epoch': 3.0})

In [12]:
results = trainer.evaluate()
print("Validation Results:", results)

Validation Results: {'eval_loss': 11.202380180358887, 'eval_runtime': 13.6487, 'eval_samples_per_second': 9.525, 'eval_steps_per_second': 1.246, 'epoch': 3.0}


In [13]:
trainer.save_model("./fine_tuned_llama_qlora")
tokenizer.save_pretrained("./fine_tuned_llama_qlora")

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


('./fine_tuned_llama_qlora/tokenizer_config.json',
 './fine_tuned_llama_qlora/special_tokens_map.json',
 './fine_tuned_llama_qlora/tokenizer.json')

In [23]:
# Install required libraries
!pip install transformers evaluate rouge_score

# Import necessary libraries
from transformers import pipeline
import evaluate

# Load the fine-tuned model and tokenizer
model_path = "./fine_tuned_llama_qlora"  # Path to your fine-tuned model
pipe = pipeline("text-generation", model=model_path, tokenizer=model_path, device=0)

# Load the ROUGE metric
rouge_metric = evaluate.load("rouge")

def test_user_query(user_query, dataset):
    """
    Generate prediction for user query and compare with the closest ground truth in the dataset.
    """
    # Generate prediction from the fine-tuned model
    generated_output = pipe(user_query, max_length=128, truncation=True)[0]["generated_text"]

    # Find the closest ground truth in the dataset
    dataset_outputs = [example["output"] for example in dataset]
    rouge_scores = []

    for gt in dataset_outputs:
        rouge_result = rouge_metric.compute(predictions=[generated_output], references=[gt])
        # Check if rougeL is a dictionary before accessing fmeasure
        if isinstance(rouge_result["rougeL"], dict):
            rouge_scores.append(rouge_result["rougeL"]["fmeasure"])
        else:
            # Handle the case where rougeL is a scalar (e.g., 0.0)
            rouge_scores.append(rouge_result["rougeL"])

    # Identify the closest ground truth based on the highest ROUGE-L score
    best_index = rouge_scores.index(max(rouge_scores))
    closest_ground_truth = dataset_outputs[best_index]

    # Print comparison
    print(f"User Query: {user_query}")
    print(f"Generated Prediction: {generated_output}")
    print(f"Closest Ground Truth: {closest_ground_truth}")
    print(f"ROUGE-L Score with Closest Ground Truth: {rouge_scores[best_index]:.4f}")

    return generated_output, closest_ground_truth, rouge_scores[best_index]

# Example usage with a user query
user_query = "What was the purpose of the application made by the Respondent wife in case FCMC 179 / 2018?"

# Test the user query against the validation dataset
generated_output, closest_ground_truth, rouge_score = test_user_query(user_query, val_data)

# Print the results
print("\n--- Summary ---")
print(f"Generated Output: {generated_output}")
print(f"Closest Ground Truth: {closest_ground_truth}")
print(f"ROUGE-L Score: {rouge_score:.4f}")


Device set to use cuda:0


User Query: What was the purpose of the application made by the Respondent wife in case FCMC 179 / 2018?
Generated Prediction: What was the purpose of the application made by the Respondent wife in case FCMC 179 / 2018? The Respondent wife is the daughter of the deceased husband of the petitioner wife. The Respondent wife was born in 1980 and she was the only daughter of the deceased husband of the petitioner wife. The deceased husband of the petitioner wife was the only son of the petitioner husband and the deceased husband of the petitioner wife. The deceased husband of the petitioner wife was the sole bread winner of the family and he was the only bread winner of the family. The Respondent wife was the only daughter of the deceased
Closest Ground Truth: The general principles demand that the court regards the best interests of the minor as the first and paramount consideration. Due consideration should also be given to the views of the minor, considering the age and understanding of